In [1]:
import csv
import pandas as pd
import re
import numpy as np

In [81]:
rawdf = pd.read_csv('C:\\Users\\mhsteiner\\Documents\\Python\\US.txt', delimiter = '\t', dtype = 'str', header = None)

In [85]:
df = rawdf.loc[:,[1,10,7,14]]
df.columns = ['city', 'state', 'code', 'pop']
df = df[df['code'].str.startswith('PPL', na=False)]


,city,state,code,pop
110,Amchitka,AK,PPL,0
199,Sunik (historical),AK,PPLQ,0
358,Attu,AK,PPL,0
516,Ugiatok (historical),AK,PPLQ,0
521,Siksatok (historical),AK,PPLQ,0
573,Navy Town,AK,PPL,0
667,Ugtikun (historical),AK,PPLQ,0
741,Ugtumuk (historical),AK,PPLQ,0
779,Ukashik (historical),AK,PPLQ,0
814,Bauxite,AL,PPL,0


In [2]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

lastcode = set([x[1] for x in list(states.keys())])

In [86]:
def state_check(code):
    name = states[code][:-1].upper().replace(' ', '') 
    inNameCode = []
    if len(name) % 2 == 0:
        for i in range(0, len(name), 2):
            inNameCode.append(name[i:i+2])
    elif name[0] in lastcode:
        for i in range(1,len(name), 2):
            inNameCode.append(name[i:i+2])
    else:
        return False
    if set(inNameCode).issubset(states.keys()):
        return True
    else:
        return False
    
def city_check(city,state):
    city = city.upper().replace(' ', '')[:-1]
    state = states[state].upper().replace(' ', '')[:-1]
    cityState = city + state
    inNameCode = []
    if len(cityState) % 2 == 0:
        for i in range(0, len(cityState), 2):
            inNameCode.append(cityState[i:i+2])
    return(True if set(inNameCode).issubset(states.keys()) and len(inNameCode) > 0 else False)        

In [99]:

goodStates2 = [state for state in df.state.unique() if state_check(state)]
limdf = df[df['state'].isin(goodStates2)]


,city,state,code,pop
2259,Eugene,FL,PPL,0
2329,Bardin,FL,PPL,0
2331,Anona,FL,PPL,0
2337,Beacon Hills,FL,PPL,0
2347,Blocker,FL,PPL,0
2365,Conant,FL,PPL,0
2371,Dune Allen Beach,FL,PPL,0
2375,Cromanton,FL,PPL,0
2399,Hollywood Beach Gardens,FL,PPL,0
2401,Happy Valley,FL,PPL,0


In [94]:
import csv
city = []
with open('test.csv') as file:
    x = csv.DictReader(file)
    for i in x:
        row = [i['city'], i['state'], i['pop']]
        city.append(row)


In [90]:
good = []
good = [c for c in city if city_check(*c[:2]) and c not in good and int(c[2]) > 0 ]

In [91]:
good

       


[['Cocoa', 'FL', '17711'],
 ['Gandy', 'FL', '2031'],
 ['Manalapan', 'FL', '449'],
 ['Memphis', 'FL', '7848'],
 ['Orlando', 'FL', '270934'],
 ['Rio', 'FL', '965'],
 ['Mack', 'OH', '11585'],
 ['Casco', 'ME', '587'],
 ['Denmark', 'ME', '1044'],
 ['Paris', 'ME', '5073'],
 ['Wyman', 'ME', '73'],
 ['Alden', 'NY', '2589'],
 ['Concord', 'NY', '8857'],
 ['Florida', 'NY', '2899'],
 ['Menands', 'NY', '3998'],
 ['Scarsdale', 'NY', '17885'],
 ['Victory', 'NY', '548']]

In [96]:
del(goodStates)

[]